In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [11]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [12]:
import netCDF4 as nc
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import numpy as np

import gsw
import arrow
import glob
import cmocean as cm
import matplotlib.pyplot as plt

from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

## Purpose of notebook

Here I'm going to take the DIC, TA, S, and T we've found in, eg, carp_timeseries_2013.nc, and calculate the pH, $\Omega_A$, and $\Omega_C$ with them. The python package I use for this is called [mocsy](http://ocmip5.ipsl.jussieu.fr/mocsy/), and it's relatively fancy - and installing it on your own computer can be a bit of a pain. If you have access to the Waterhole cluster, you can just use the path to the installation that I listed above. Below I've written a wrapper code for it (oned_moxy). 




In [13]:
t_nc = nc.Dataset('./EXTRACTED_timeseries/carp_timeseries_2016.nc')
t_str = 'pHOm_timeseries_2016.nc'

#### TJSJ wrapper code for mocsy

In [14]:
def oned_moxy(tsal, ttemp, tdic, tta, pres_atm, depth_this):

    size_box = np.shape(tdic)
    size_0 = size_box[0]
    size_1= size_box[1]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-3
    tdra = np.ravel(tdic) * 1e-3
    tzero = np.zeros_like(tsra)
    tpressure = np.zeros_like(tsra)
    tdepth = np.zeros_like(tsra)
    tdepth[:] = depth_this
     
    tpressure[:] = pres_atm
    tzero = tpressure * 0 
        
    tsra_psu = tsra*35/35.16504
    ttera_is = gsw.t_from_CT(tsra,ttera,tzero)

    response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tdepth, lat=tzero, 
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pHr = pH.reshape(size_0,size_1)
    OmAr = OmegaA.reshape(size_0,size_1)
    OmCr = OmegaC.reshape(size_0,size_1)
    pco2r = pco2.reshape(size_0,size_1)
    
    return pHr, OmAr, OmCr, pco2r

#### TJSJ let's make a dict in which to store the OmA and pH timeseries

In [15]:
PLACES_withdat = {
'Deep Bay': {
'NEMO grid ji': (599, 126),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])

},
    
'Southern Baynes': {
'NEMO grid ji': (602, 127),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Northern Baynes': {
'NEMO grid ji': (646, 127),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Fanny Bay': {
'NEMO grid ji': (614, 120),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Maple Bay': {
'NEMO grid ji': (392, 213),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Salt Spring': {
'NEMO grid ji': (386, 218),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Nanoose Bay': {
'NEMO grid ji': (517, 190),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Lasqueti Island': {
'NEMO grid ji': (586, 195),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])
},

'Main SoG': {
'NEMO grid ji': (450, 253),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])

},

'Cortes/Marina': {
'NEMO grid ji': (732, 157),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Lund/Desolation Sound': {
'NEMO grid ji': (702, 187),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

'Mouth of Okeover': {
'NEMO grid ji': (726, 192),
    'OmA_timeseries':np.zeros([40,365]),
    'OmC_timeseries':np.zeros([40,365]),
    'pH_timeseries':np.zeros([40,365])},

}

list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', \
               'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay',\
               'Lasqueti Island', 'Main SoG', 'Cortes/Marina', \
               'Lund/Desolation Sound', 'Mouth of Okeover']


#### TJSJ let's open the .nc we made in part B, extract the DIC, TA, S, T, variables and run mocsy on them to get pH and Omega A, then store these in the dictionary we just made

In [16]:
for i in range(0,len(list_places)):
    stn = list_places[i]

    DIC = t_nc[stn]['DIC_timeseries'][:,:]
    TA = t_nc[stn]['TA_timeseries'][:,:]
    TEMP = t_nc[stn]['T_timeseries'][:,:]
    SAL = t_nc[stn]['S_timeseries'][:,:]

    pHr, OmAr, OmCr, pco2r =oned_moxy(SAL, TEMP, DIC, TA, 1, 0)

    PLACES_withdat[stn]['OmA_timeseries'] = OmAr
    PLACES_withdat[stn]['OmC_timeseries'] = OmCr
    PLACES_withdat[stn]['pH_timeseries'] = pHr


In [17]:


f = nc.Dataset(t_str, 'w')
g = f.createGroup('Deep Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['pH_timeseries'][:]


g = f.createGroup('Southern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['pH_timeseries'][:]


g = f.createGroup('Northern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['pH_timeseries'][:]


g = f.createGroup('Fanny Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['pH_timeseries'][:]


g = f.createGroup('Maple Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['pH_timeseries'][:]

g = f.createGroup('Salt Spring')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['pH_timeseries'][:]


g = f.createGroup('Nanoose Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['pH_timeseries'][:]

g = f.createGroup('Lasqueti Island')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['pH_timeseries'][:]

g = f.createGroup('Main SoG')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['pH_timeseries'][:]


g = f.createGroup('Cortes/Marina')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['pH_timeseries'][:]


g = f.createGroup('Lund/Desolation Sound')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['pH_timeseries'][:]


g = f.createGroup('Mouth of Okeover')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('OmA_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['OmA_timeseries'][:]
ts = g.createVariable('OmC_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['OmC_timeseries'][:]
ts = g.createVariable('pH_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['pH_timeseries'][:]



f.close()

In [18]:
print('done, yo')

done, yo
